In [146]:
import numpy as np
import pandas as pd
import xgboost as xgb
total=pd.read_csv("./Final2.csv")

In [147]:
property_features = ['id','price_doc',
                'full_sq',
                'life_sq',
                'num_room',
                'sport_count_5000',
                'trc_count_5000',
                'zd_vokzaly_avto_km',
                'sadovoe_km',
                'kremlin_km',
                'bulvar_ring_km',
                'ttk_km',
                'office_sqm_5000',
                'nuclear_reactor_km',
                'cafe_count_5000_price_1000',
                'stadium_km',
                'kitch_sq',
                'theater_km',
                'swim_pool_km',
                'catering_km',
                'workplaces_km',
                'school_education_centers_raion',
                'big_church_km',
                'metro_min_avto',
                'public_healthcare_km',
                'big_road2_km',
                'ratio_brick',
                'green_part_5000',
                'ice_rink_km',
                'additional_education_km',
                'school_km',
                'ratio_1920',
                'railroad_station_avto_min','train_or_test','OKRUG','log_log_linear','Stack']

total1=total[property_features]

In [148]:
from sklearn import preprocessing
le_state = preprocessing.LabelEncoder()

In [149]:
le_state.fit(total1["OKRUG"])

LabelEncoder()

In [150]:
total1["OKRUG"] = le_state.transform(total1["OKRUG"])

/Users/mac/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [152]:
a = pd.get_dummies(total1.OKRUG)
L = range(0, len(a.columns)) 
a.columns = ["OKRUG"+str(x) for x in L]
total1 = pd.concat([total1,a], axis=1)

In [153]:
total1.drop(['OKRUG'],inplace = True, axis=1)

In [154]:
test = total1[total1['train_or_test']=='test']
train = total1[total1["train_or_test"]=="train"]

In [155]:
test = test.drop(["id","train_or_test","price_doc"], axis= 1)

In [156]:
train["log_price"] = train["price_doc"].apply(np.log)

/Users/mac/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [157]:
train = train.drop(["id", "train_or_test",'price_doc'], axis = 1)

In [158]:
train.shape

(30464, 58)

In [159]:
test.shape

(7662, 57)

In [160]:
train.columns.values

array(['full_sq', 'life_sq', 'num_room', 'sport_count_5000',
       'trc_count_5000', 'zd_vokzaly_avto_km', 'sadovoe_km', 'kremlin_km',
       'bulvar_ring_km', 'ttk_km', 'office_sqm_5000', 'nuclear_reactor_km',
       'cafe_count_5000_price_1000', 'stadium_km', 'kitch_sq',
       'theater_km', 'swim_pool_km', 'catering_km', 'workplaces_km',
       'school_education_centers_raion', 'big_church_km', 'metro_min_avto',
       'public_healthcare_km', 'big_road2_km', 'ratio_brick',
       'green_part_5000', 'ice_rink_km', 'additional_education_km',
       'school_km', 'ratio_1920', 'railroad_station_avto_min',
       'log_log_linear', 'Stack', 'OKRUG0', 'OKRUG1', 'OKRUG2', 'OKRUG3',
       'OKRUG4', 'OKRUG5', 'OKRUG6', 'OKRUG7', 'OKRUG8', 'OKRUG9',
       'OKRUG10', 'OKRUG11', 'OKRUG0', 'OKRUG1', 'OKRUG2', 'OKRUG3',
       'OKRUG4', 'OKRUG5', 'OKRUG6', 'OKRUG7', 'OKRUG8', 'OKRUG9',
       'OKRUG10', 'OKRUG11', 'log_price'], dtype=object)

In [162]:
property_features_1 = [
                'full_sq',
                'life_sq',
                'num_room',
                'sport_count_5000',
                'trc_count_5000',
                'zd_vokzaly_avto_km',
                'sadovoe_km',
                'kremlin_km',
                'bulvar_ring_km',
                'ttk_km',
                'office_sqm_5000',
                'nuclear_reactor_km',
                'cafe_count_5000_price_1000',
                'stadium_km',
                'kitch_sq',
                'theater_km',
                'swim_pool_km',
                'catering_km',
                'workplaces_km',
                'school_education_centers_raion',
                'big_church_km',
                'metro_min_avto',
                'public_healthcare_km',
                'big_road2_km',
                'ratio_brick',
                'green_part_5000',
                'ice_rink_km',
                'additional_education_km',
                'school_km',
                'ratio_1920',
                'railroad_station_avto_min',
                'log_log_linear','Stack']

In [163]:
X_train = train[property_features_1].values
Y_train = train['log_price'].values
X_test = test[property_features_1].values

size_ = 7000
X_train_sub, Y_train_sub = X_train[:-size_],  Y_train[:-size_]
X_val, Y_val = X_train[-size_:],  Y_train[-size_:]

In [164]:
dtrain = xgb.DMatrix(X_train, 
                    Y_train, 
                    feature_names=property_features_1)
dtrain_sub = xgb.DMatrix(X_train_sub, 
                        Y_train_sub, 
                        feature_names=property_features_1)
d_val = xgb.DMatrix(X_val, 
                    Y_val, 
                    feature_names=property_features_1)
dtest = xgb.DMatrix(X_test, 
                    feature_names=property_features_1)

In [165]:
# hyperparameters
xgb_params = {
    'eta': 0.02,
    'max_depth': 5,
    'subsample': .8,
    'colsample_bytree': 0.7,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'silent': 1
}

# Uncomment to tune XGB `num_boost_rounds`
sub_model = xgb.train(xgb_params, 
                      dtrain_sub, 
                      num_boost_round=2000,
                      evals=[(d_val, 'val')],
                      early_stopping_rounds=20, 
                      verbose_eval=20)

[0]	val-rmse:14.9181
Will train until val-rmse hasn't improved in 20 rounds.
[20]	val-rmse:9.97452
[40]	val-rmse:6.6718
[60]	val-rmse:4.46754
[80]	val-rmse:2.99893
[100]	val-rmse:2.02253
[120]	val-rmse:1.37717
[140]	val-rmse:0.954807
[160]	val-rmse:0.684105
[180]	val-rmse:0.518595
[200]	val-rmse:0.422769
[220]	val-rmse:0.372523
[240]	val-rmse:0.346062
[260]	val-rmse:0.332841
[280]	val-rmse:0.326513
[300]	val-rmse:0.322696
[320]	val-rmse:0.320549
[340]	val-rmse:0.319225
[360]	val-rmse:0.317739
[380]	val-rmse:0.316905
[400]	val-rmse:0.316031
[420]	val-rmse:0.315389
[440]	val-rmse:0.314864
[460]	val-rmse:0.314007
[480]	val-rmse:0.313404
[500]	val-rmse:0.312853
[520]	val-rmse:0.312219
[540]	val-rmse:0.311624
[560]	val-rmse:0.311187
[580]	val-rmse:0.3107
[600]	val-rmse:0.31016
[620]	val-rmse:0.309601
[640]	val-rmse:0.30909
[660]	val-rmse:0.308619
[680]	val-rmse:0.308082
[700]	val-rmse:0.307794
[720]	val-rmse:0.307332
[740]	val-rmse:0.307041
[760]	val-rmse:0.306707
[780]	val-rmse:0.306377
[8

In [166]:
full_model = xgb.train(xgb_params,
                       dtrain, 
                       num_boost_round=sub_model.best_iteration,
                       verbose_eval=20)

In [167]:
log_y_pred = full_model.predict(dtest)
y_pred = np.exp(log_y_pred) - 1

In [171]:
submit = pd.DataFrame({'id': np.array(test.index), 'price_doc': y_pred})
submit.to_csv('submission_xg_boost_diff.csv', index=False)

In [169]:
submit.shape

(7662, 2)